<a href="https://colab.research.google.com/github/brrikcy/deep-learning/blob/main/DL_EXP_04_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

Enabling TPU

In [ ]:
try:
 tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # Get TPU resolver
 print('Running on TPU ', tpu.master())
except ValueError:
 print('TPU not found.')
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# Define TPU
strategy = tf.distribute.TPUStrategy(tpu)

Running on TPU  


Loading MNIST dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Preprocessing

In [ ]:
#normalization
x_train, x_test = x_train / 255.0, x_test / 255.0

#one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# CNN model with L1 regularization

In [ ]:
# Initialize Sequential model_l1
model_l1 = models.Sequential()

# Convolutional layer with L1 regularization and Gaussian Noise
model_l1.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                        kernel_regularizer=regularizers.l1(1e-5), input_shape=(28, 28, 1)))  # Adjusted input shape
model_l1.add(layers.GaussianNoise(0.1))  # Add Gaussian noise

# Second convolutional layer with L1 regularization
model_l1.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                        kernel_regularizer=regularizers.l1(1e-5)))
model_l1.add(layers.MaxPooling2D((2, 2)))
model_l1.add(layers.Dropout(0.25))  # Dropout for regularization

# Third convolutional layer
model_l1.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model_l1.add(layers.MaxPooling2D((2, 2)))

# Flatten the network output
model_l1.add(layers.Flatten())

# Dense layer with L1 regularization and Dropout
model_l1.add(layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l1(1e-5)))
model_l1.add(layers.Dropout(0.5))  # Dropout for regularization

# Output layer
model_l1.add(layers.Dense(10, activation='softmax'))  # 10 output classes for digits 0-9


NameError: name 'models' is not defined

# CNN model with L2 regularization

In [ ]:
# Initialize Sequential model_l2
model_l2 = models.Sequential()

# Convolutional layer with L1 regularization and Gaussian Noise
model_l2.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                        kernel_regularizer=regularizers.l2(1e-4), input_shape=(28, 28, 1)))  # Adjusted input shape
model_l2.add(layers.GaussianNoise(0.1))  # Add Gaussian noise

# Second convolutional layer with L1 regularization
model_l2.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                        kernel_regularizer=regularizers.l2(1e-4)))
model_l2.add(layers.MaxPooling2D((2, 2)))
model_l2.add(layers.Dropout(0.25))  # Dropout for regularization

# Third convolutional layer
model_l2.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model_l2.add(layers.MaxPooling2D((2, 2)))

# Flatten the network output
model_l2.add(layers.Flatten())

# Dense layer with L1 regularization and Dropout
model_l2.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
model_l2.add(layers.Dropout(0.5))  # Dropout for regularization

# Output layer
model_l2.add(layers.Dense(10, activation='softmax'))  # 10 output classes for digits 0-9


Model compilation

In [ ]:
models=[model_l1,model_l2]
for model in models:
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Defining EarlyStopping callback

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

Train the models

In [ ]:
for model in models:
  history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping])


Epoch 1/10
938/938 [==============================] - 21s 21ms/step - loss: 0.4608 - accuracy: 0.8675 - val_loss: 0.1031 - val_accuracy: 0.9819
Epoch 2/10
938/938 [==============================] - 20s 21ms/step - loss: 0.1688 - accuracy: 0.9650 - val_loss: 0.0878 - val_accuracy: 0.9874
Epoch 3/10
938/938 [==============================] - 19s 21ms/step - loss: 0.1373 - accuracy: 0.9747 - val_loss: 0.0816 - val_accuracy: 0.9877
Epoch 4/10
938/938 [==============================] - 19s 20ms/step - loss: 0.1200 - accuracy: 0.9798 - val_loss: 0.0795 - val_accuracy: 0.9897
Epoch 5/10
938/938 [==============================] - 19s 21ms/step - loss: 0.1106 - accuracy: 0.9820 - val_loss: 0.0786 - val_accuracy: 0.9913
Epoch 6/10
938/938 [==============================] - 19s 21ms/step - loss: 0.1056 - accuracy: 0.9840 - val_loss: 0.0790 - val_accuracy: 0.9905
Epoch 7/10
938/938 [==============================] - 19s 20ms/step - loss: 0.1017 - accuracy: 0.9852 - val_loss: 0.0794 - val_accuracy:

Evaluating the model

In [ ]:
test_loss, test_acc = model_l1.evaluate(x_test, y_test)
print(f"Test accuracy with L1 regularization: {test_acc*100} %")

test_loss, test_acc = model_l2.evaluate(x_test, y_test)
print(f"Test accuracy with L2 regularization: {test_acc*100} %")

313/313 [==============================] - 2s 5ms/step - loss: 0.0731 - accuracy: 0.9932
Test accuracy with L1 regularization: 99.32000041007996 %
313/313 [==============================] - 2s 5ms/step - loss: 0.0517 - accuracy: 0.9934
Test accuracy with L2 regularization: 99.3399977684021 %


In [ ]:
# Access the GaussianNoise layer
gaussian_noise_layer_l1 = model_l1.layers[1]
gaussian_noise_layer_l2 = model_l2.layers[1]


print(f"gaussian noise in l1 model:{gaussian_noise_layer_l1.stddev}")
print(f"gaussian noise in l2 model:{gaussian_noise_layer_l2.stddev}")

gaussian noise in l1 model:0.1
gaussian noise in l2 model:0.1
